# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,west island,-12.1568,96.8225,26.99,78,100,4.63,CC,1710434429
1,1,bethel,41.3712,-73.4140,20.91,40,0,2.06,US,1710434332
2,2,vorgashor,67.5833,63.9500,-24.83,100,74,1.66,RU,1710434429
3,3,waitangi,-43.9535,-176.5597,12.31,67,36,2.32,NZ,1710434429
4,4,priozersk,61.0393,30.1291,5.05,83,100,8.67,RU,1710434429


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Create the map plot
map_plot = city_data_df.hvplot.points(
    x='Lng',
    y='Lat',
    geo=True,
    size='Humidity',  # Use humidity for the size of the points
    color='City',     # Use city names for different colors
    alpha = 0.5,      # Add transparency to colors
    tiles="OSM",
    hover_cols=["City", "Humidity"],  # Add city and humidity info in hover
    title='Cities and Humidity',  # Set the title of the plot
    width=1000, height=600  # Set the width and height of the plot
)

# Display the map plot
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values

# Define ideal weather conditions
max_temp_lower = 21
max_temp_upper = 27
max_wind_speed = 4.5
max_cloudiness = 0

# Filter the DataFrame based on ideal weather conditions
ideal_weather_df = city_data_df[
    (city_data_df['Max Temp'] > max_temp_lower) & 
    (city_data_df['Max Temp'] < max_temp_upper) & 
    (city_data_df['Wind Speed'] < max_wind_speed) & 
    (city_data_df['Cloudiness'] == max_cloudiness)
]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
66,66,fort bragg,35.1390,-79.0060,25.51,30,0,2.57,US,1710434175
81,81,kill devil hills,36.0307,-75.6760,24.00,28,0,1.54,US,1710434283
88,88,pigeon forge,35.7884,-83.5543,23.81,49,0,3.09,US,1710434435
109,109,tura,25.5198,90.2201,21.83,25,0,1.08,IN,1710434437
159,159,ormond-by-the-sea,29.3491,-81.0665,26.15,50,0,2.57,US,1710434441
198,198,'ain el turk,35.7438,-0.7693,22.50,60,0,4.12,DZ,1710434447
211,211,port elizabeth,-33.9180,25.5701,22.16,70,0,3.60,ZA,1710434357
219,219,kailua-kona,19.6406,-155.9956,21.98,80,0,2.57,US,1710434263
222,222,atlantic city,39.3643,-74.4229,24.54,52,0,2.57,US,1710434448
257,257,crane,31.3974,-102.3501,21.40,20,0,2.33,US,1710434450


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

hotel_df = ideal_weather_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
66,fort bragg,US,35.1390,-79.0060,30,
81,kill devil hills,US,36.0307,-75.6760,28,
88,pigeon forge,US,35.7884,-83.5543,49,
109,tura,IN,25.5198,90.2201,25,
159,ormond-by-the-sea,US,29.3491,-81.0665,50,
198,'ain el turk,DZ,35.7438,-0.7693,60,
211,port elizabeth,ZA,-33.9180,25.5701,70,
219,kailua-kona,US,19.6406,-155.9956,80,
222,atlantic city,US,39.3643,-74.4229,52,
257,crane,US,31.3974,-102.3501,20,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    #"radius": radius,
    "categories": "accommodation",
    "limit": 1,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    # Convert the API response to JSON format
    name_address = name_address.json()
        
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
fort bragg - nearest hotel: Airborne Inn Lodging
kill devil hills - nearest hotel: Mariner Days Inn & Suites
pigeon forge - nearest hotel: Riverside Towers
tura - nearest hotel: Hotel Polo Orchid
ormond-by-the-sea - nearest hotel: Econo Inn - Motel Ormond Beach
'ain el turk - nearest hotel: Hôtel Chems
port elizabeth - nearest hotel: Waterford Hotel
kailua-kona - nearest hotel: Kona Seaside Hotel
atlantic city - nearest hotel: Knights Inn Atlantic City
crane - nearest hotel: No hotel found
freeport - nearest hotel: Yankee Clipper Motel
geraldton - nearest hotel: Ocean Centre Hotel
katol - nearest hotel: No hotel found
tres arroyos - nearest hotel: Andrea Hotel
dhandhuka - nearest hotel: No hotel found
greenville - nearest hotel: Quality Inn
jaisinghnagar - nearest hotel: No hotel found
marshall - nearest hotel: Wyndham Garden Lake-Guntersville
cam ranh - nearest hotel: Khách sạn Thái Bình
al ghayzah - nearest hotel: فندق تاج العرب
emerald - nearest hotel: Emerald 

,City,Country,Lat,Lng,Humidity,Hotel Name
66,fort bragg,US,35.1390,-79.0060,30,Airborne Inn Lodging
81,kill devil hills,US,36.0307,-75.6760,28,Mariner Days Inn & Suites
88,pigeon forge,US,35.7884,-83.5543,49,Riverside Towers
109,tura,IN,25.5198,90.2201,25,Hotel Polo Orchid
159,ormond-by-the-sea,US,29.3491,-81.0665,50,Econo Inn - Motel Ormond Beach
198,'ain el turk,DZ,35.7438,-0.7693,60,Hôtel Chems
211,port elizabeth,ZA,-33.9180,25.5701,70,Waterford Hotel
219,kailua-kona,US,19.6406,-155.9956,80,Kona Seaside Hotel
222,atlantic city,US,39.3643,-74.4229,52,Knights Inn Atlantic City
257,crane,US,31.3974,-102.3501,20,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_hotel = hotel_df.hvplot.points(
    x='Lng',
    y='Lat',
    geo=True,
    color='City',  # Color the points by hotel name
    tiles="OSM",
    hover_cols=["City", "Hotel Name"],  # Add city and hotel name info in hover
    title='Cities with Nearest Hotel',  # Set the title of the plot
    width=1000, height=600  # Set the width and height of the plot
)

# Display the map plot
map_plot_hotel

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name)